In [85]:
from os import listdir
from os.path import splitext
from os.path import join

import skvideo
skvideo.setFFmpegPath(r'C:\Users\ion\anaconda3\envs\tf-gpu\Lib\site-packages\ffmpeg-4.3.1-2020-10-01-full_build\bin')
import skvideo.io

import json

In [3]:
# Aff-Wild2 = AW2
# AFEW 7.0  = AF7
# filenames = FN

### Aff-wild2 ###
# Directories for labels/annotations for each set
AW2_dir_train_labels = (r"D:\Aff-Wild2 Dataset\Aff-wild2\Videos\annotations\EXPR_Set\Training_Set")
AW2_dir_val_labels = (r"D:\Aff-Wild2 Dataset\Aff-wild2\Videos\annotations\EXPR_Set\Validation_Set")

# Directories for videos
AW2_dir_allvideos = r"D:\Aff-Wild2 Dataset\Aff-wild2\Videos\all_videos"

# Save the filenames in a list for each set, and split the filename from the extension
AW2_train_FN = listdir(AW2_dir_train_labels)
AW2_train_FN_split = [splitext(file) for file in AW2_train_FN]

AW2_val_FN = listdir(AW2_dir_val_labels)
AW2_val_FN_split = [splitext(file) for file in AW2_val_FN]

AW2_videos_FN = listdir(AW2_dir_allvideos)
AW2_videos_FN_split = [splitext(file) for file in AW2_videos_FN]

In [4]:
# Size of each set
print(f"The training-set contains {len(AW2_train_FN)} videos")
print(f"The validation-set contains {len(AW2_val_FN)} videos")
print(f"The test-set contains {len(AW2_videos_FN) - (len(AW2_train_FN) + len(AW2_val_FN))} videos")
print(f"The whole dataset contains {len(AW2_videos_FN)} videos")

The training-set contains 253 videos
The validation-set contains 70 videos
The test-set contains 225 videos
The whole dataset contains 548 videos


In [5]:
# Checking if the labeled filenames corresponds to the videos
train_size = 0
for i in AW2_train_FN_split:
    for j in AW2_videos_FN_split:
        if i[0] == j[0]:
#             print(i[0], ",", j[0])
            train_size+=1
    
## The results show that only 246 labels are matched to videos, while there were 253 labeled videos
## Some videos contain two persons, so the labels are for one person on the left or right
print(train_size)

246


In [6]:
# Checking if the labeled filenames corresponds to the videos (and disregarding if its on left or right)
train_size = 0
for i in AW2_train_FN_split:
    for j in AW2_videos_FN_split:
        if '_left' in i[0]:
            a = i[0].replace('_left', '')
        elif '_right' in i[0]:
            a = i[0].replace('_right', '')
        else:
            a = i[0]
        if a == j[0]:
            train_size+=1
# Labels is equal to videos now
print(f'Training dataset is indeed = {train_size} videos')

# Checking if the labeled filenames corresponds to the videos (and disregarding if its on left or right)
val_size = 0
for i in AW2_val_FN_split:
    for j in AW2_videos_FN_split:
        if '_left' in i[0]:
            a = i[0].replace('_left', '')
        elif '_right' in i[0]:
            a = i[0].replace('_right', '')
        else:
            a = i[0]
        if a == j[0]:
            val_size+=1
# Labels is equal to videos now
print(f'Validation dataset is indeed = {val_size} videos')

Training dataset is indeed = 253 videos
Validation dataset is indeed = 70 videos


### START - No need to run this anymore ###

In [99]:
# Create dictionary of sequence length of each video
seqlength = {}

for video in AW2_videos_FN:
    metadata = skvideo.io.ffprobe(join(AW2_dir_allvideos, video))
    frames = int(metadata.get('video').get('@nb_frames'))
    width = int(metadata.get('video').get('@width'))
    height = int(metadata.get('video').get('@height'))
    seqlength[video] = [frames, height, width, 3]

# Save json file with all video's names and shapes = (n_frames, width, height, colour channels)
with open('video_shapes.json', 'w') as fp:
    json.dump(seqlength, fp)

In [92]:
seqlength

{'1-30-1280x720.mp4': (10355, 720, 1280, 3),
 '10-60-1280x720.mp4': (2501, 720, 1280, 3),
 '100-29-1080x1920.mp4': (2639, 1920, 1080, 3),
 '101-30-1080x1920.mp4': (5153, 1920, 1080, 3),
 '102-30-640x360.mp4': (2983, 360, 640, 3),
 '102.avi': (4777, 360, 480, 3),
 '103-30-384x480.mp4': (771, 480, 384, 3),
 '103.avi': (26617, 360, 600, 3),
 '104-17-720x480.mp4': (2425, 480, 720, 3),
 '105-30-1280x720.mp4': (1501, 720, 1280, 3),
 '105.avi': (10076, 360, 640, 3),
 '106-30-720x1280.mp4': (1404, 1280, 720, 3),
 '106.avi': (8163, 360, 480, 3),
 '107-30-640x480.mp4': (73, 480, 640, 3),
 '107.avi': (5315, 360, 640, 3),
 '108-15-640x480.mp4': (478, 480, 640, 3),
 '108.avi': (4678, 360, 640, 3),
 '109-30-1280x720.mp4': (1045, 720, 1280, 3),
 '11-24-1920x1080.mp4': (5389, 1080, 1920, 3),
 '110-30-270x480.mp4': (9978, 480, 270, 3),
 '110.avi': (5504, 360, 640, 3),
 '111-25-1920x1080.mp4': (771, 1080, 1920, 3),
 '111.avi': (7246, 360, 640, 3),
 '112-30-640x360.mp4': (6002, 360, 640, 3),
 '112.avi': 

### END - No need to run this anymore ###

In [106]:
# Load video_shapes.json
with open('video_shapes.json', 'r') as fp:
    video_shapes = json.load(fp)

# for i in AW2_videos_FN:
#     print(video_shapes.get(i))

In [ ]:
# # Create dictionary of sequence length
# seqlength = {}

# # Save the shape information of the array for each file in the previously made dictionary
# for video in AW2_videos_FN:
#     cache = skvideo.io.vread(join(AW2_dir_allvideos, video))
#     seqlength[video] = cache.shape
# print(seqlength)

In [104]:
x.get('video')

OrderedDict([('@index', '0'),
             ('@codec_name', 'h264'),
             ('@codec_long_name', 'H.264 / AVC / MPEG-4 AVC / MPEG-4 part 10'),
             ('@profile', 'High'),
             ('@codec_type', 'video'),
             ('@codec_time_base', '1/60'),
             ('@codec_tag_string', 'avc1'),
             ('@codec_tag', '0x31637661'),
             ('@width', '640'),
             ('@height', '360'),
             ('@coded_width', '640'),
             ('@coded_height', '368'),
             ('@closed_captions', '0'),
             ('@has_b_frames', '2'),
             ('@sample_aspect_ratio', '1:1'),
             ('@display_aspect_ratio', '16:9'),
             ('@pix_fmt', 'yuv420p'),
             ('@level', '30'),
             ('@chroma_location', 'left'),
             ('@refs', '1'),
             ('@is_avc', 'true'),
             ('@nal_length_size', '4'),
             ('@r_frame_rate', '30/1'),
             ('@avg_frame_rate', '30/1'),
             ('@time_base', '1/15360')

In [110]:
video_shapes

{'1-30-1280x720.mp4': [10355, 720, 1280, 3],
 '10-60-1280x720.mp4': [2501, 720, 1280, 3],
 '100-29-1080x1920.mp4': [2639, 1920, 1080, 3],
 '101-30-1080x1920.mp4': [5153, 1920, 1080, 3],
 '102-30-640x360.mp4': [2983, 360, 640, 3],
 '102.avi': [4777, 360, 480, 3],
 '103-30-384x480.mp4': [771, 480, 384, 3],
 '103.avi': [26617, 360, 600, 3],
 '104-17-720x480.mp4': [2425, 480, 720, 3],
 '105-30-1280x720.mp4': [1501, 720, 1280, 3],
 '105.avi': [10076, 360, 640, 3],
 '106-30-720x1280.mp4': [1404, 1280, 720, 3],
 '106.avi': [8163, 360, 480, 3],
 '107-30-640x480.mp4': [73, 480, 640, 3],
 '107.avi': [5315, 360, 640, 3],
 '108-15-640x480.mp4': [478, 480, 640, 3],
 '108.avi': [4678, 360, 640, 3],
 '109-30-1280x720.mp4': [1045, 720, 1280, 3],
 '11-24-1920x1080.mp4': [5389, 1080, 1920, 3],
 '110-30-270x480.mp4': [9978, 480, 270, 3],
 '110.avi': [5504, 360, 640, 3],
 '111-25-1920x1080.mp4': [771, 1080, 1920, 3],
 '111.avi': [7246, 360, 640, 3],
 '112-30-640x360.mp4': [6002, 360, 640, 3],
 '112.avi': 